In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### image dataset load

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

data_dir = r'/content/drive/My Drive/기업프로젝트-라젠/data/40, 320'

IMAGE_SIZE = (299,299)
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

Found 13120 files belonging to 320 classes.
Using 10496 files for training.
Found 13120 files belonging to 320 classes.
Using 2624 files for validation.


In [ ]:
print(train_ds)
print(val_ds)

<BatchDataset shapes: ((None, 299, 299, 3), (None,)), types: (tf.float32, tf.int32)>
<BatchDataset shapes: ((None, 299, 299, 3), (None,)), types: (tf.float32, tf.int32)>


### 데이터 정규화

In [ ]:
from tensorflow.keras import layers

# 0~1 사이의 값을 가지도록 조정 
def normalization_dataset(dataset):
    normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
    normalized_ds = dataset.map(lambda x, y: (normalization_layer(x), y))
    return normalized_ds 

train_ds = normalization_dataset(train_ds)
val_ds = normalization_dataset(val_ds)
print(train_ds)
print(val_ds)

<MapDataset shapes: ((None, 299, 299, 3), (None,)), types: (tf.float32, tf.int32)>
<MapDataset shapes: ((None, 299, 299, 3), (None,)), types: (tf.float32, tf.int32)>


### 성능을위한 데이터 세트 구성

In [ ]:
# cache() 첫 번째 에포크 동안 디스크에서로드 된 이미지를 메모리에 보관합니다. 
# prefetch 학습 중에 데이터 전처리 및 모델 실행과 겹칩니다

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


### tensorflow module load

In [ ]:
import tensorflow_hub as hub
import tensorflow_addons as tfa
 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Lambda, MaxPooling2D, Conv2D
from tensorflow.keras.optimizers import Adam, SGD, Adagrad

from tensorflow.keras.callbacks import EarlyStopping

### 모델 load, train

In [ ]:
# mobilenet_v2
hub_url = "https://tfhub.dev/google/imagenet/inception_v3/classification/4"
hub_layer = hub.KerasLayer(hub_url,
                           input_shape=IMAGE_SIZE + (3,),
                           trainable=False)

# model구축
# triplets 128벡터값을 위한 dense출력값을 128로 설정, activation = None
# deep architecture이후 l2정규화
model = Sequential()
model.add(hub_layer)
model.add(Dense(128, activation=None))
model.add(Lambda(lambda x: tf.math.l2_normalize(x, axis=1, epsilon=1e-10)))  

model.summary()

# 모델 complie
# adam = Adam(lr=0.001, decay=1e-6)
# model.compile(optimizer=adam, loss=tfa.losses.TripletSemiHardLoss())

SGD = SGD(lr=0.05)
model.compile(optimizer=SGD, loss=tfa.losses.TripletSemiHardLoss())

# Adagrad = Adagrad(lr=0.01, epsilon=1e-6)
# model.compile(optimizer=Adagrad, loss=tfa.losses.TripletSemiHardLoss())

# earlystopping
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

# 모델 학습
history = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs=1000,
                    # callbacks=[early_stopping]
                    )  

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 1001)              23853833  
_________________________________________________________________
dense_2 (Dense)              (None, 128)               128256    
_________________________________________________________________
lambda_2 (Lambda)            (None, 128)               0         
Total params: 23,982,089
Trainable params: 128,256
Non-trainable params: 23,853,833
_________________________________________________________________
Epoch 1/1000
328/328 [==============================] - ETA: 0s - loss: nanWARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0152s vs `on_test_batch_end` time: 0.0595s). Check your callbacks.


328/328 [==============================] - 32s 98ms/step - loss: nan - val_loss: nan
Epoch 2/1000
328/328 [==============================] - 31s 95ms/step - loss: nan - val_loss: nan
Epoch 3/1000
328/328 [==============================] - 31s 95ms/step - loss: nan - val_loss: nan
Epoch 4/1000
328/328 [==============================] - 31s 95ms/step - loss: nan - val_loss: nan
Epoch 5/1000
328/328 [==============================] - 31s 95ms/step - loss: nan - val_loss: nan
Epoch 6/1000
328/328 [==============================] - 31s 95ms/step - loss: nan - val_loss: nan
Epoch 7/1000
328/328 [==============================] - 31s 95ms/step - loss: nan - val_loss: nan
Epoch 8/1000
328/328 [==============================] - 31s 95ms/step - loss: nan - val_loss: nan
Epoch 9/1000
328/328 [==============================] - 31s 95ms/step - loss: nan - val_loss: nan
Epoch 10/1000
328/328 [==============================] - 31s 95ms/step - loss: nan - val_loss: nan
Epoch 11/1000
328/328 [=========

KeyboardInterrupt: ignored

In [ ]:
model.save('/content/drive/My Drive/기업프로젝트-라젠/model/inception_v3-2.h5')

### load model

In [ ]:

print(len(inception_v3.layers) * 0.7)

219.1


In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.applications import InceptionV3

inception_v3 = InceptionV3()


# print(len(inception_v3.layers) * 0.2)

for layer in inception_v3.layers[219:-1]:
    layer.trainable = True

for layer in inception_v3.layers[219:-1]:
    if 'kernel' in layer.__dict__:
        kernel_shape = np.array(layer.get_weights()).shape
        layer.set_weights(tf.random.normal(kernel_shape, 0, 1))



import tensorflow_addons as tfa

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Lambda
from tensorflow.keras.optimizers import Adam

x = inception_v3.layers[-2].output
x2 = Dense(128, activation=None)(x)
outputs = Lambda(lambda x: tf.math.l2_normalize(x, axis=1, epsilon=1e-12))(x2)
model = Model(inputs=inception_v3.input,
              outputs=outputs)

adam = Adam(lr=0.001, decay=1e-6)
# sgd = SGD(lr=0.05)
model.compile(optimizer=adam,
              loss=tfa.losses.TripletSemiHardLoss())

history = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs=1000,
                    # callbacks=[early_stopping]
                    )  

Epoch 1/1000
153/328 [============>.................] - ETA: 42s - loss: nan

KeyboardInterrupt: ignored